## **Part I**

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Convolutional neural network (3 Convolutional layers + 1 Fully connected layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


100%|██████████| 170498071/170498071 [00:13<00:00, 12715387.02it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/10], Step [100/782], Loss: 1.6014
Epoch [1/10], Step [200/782], Loss: 1.7285
Epoch [1/10], Step [300/782], Loss: 1.2765
Epoch [1/10], Step [400/782], Loss: 1.2570
Epoch [1/10], Step [500/782], Loss: 1.2392
Epoch [1/10], Step [600/782], Loss: 0.9949
Epoch [1/10], Step [700/782], Loss: 1.2892
Epoch [2/10], Step [100/782], Loss: 1.2479
Epoch [2/10], Step [200/782], Loss: 1.0150
Epoch [2/10], Step [300/782], Loss: 1.0945
Epoch [2/10], Step [400/782], Loss: 0.8884
Epoch [2/10], Step [500/782], Loss: 0.7958
Epoch [2/10], Step [600/782], Loss: 1.0781
Epoch [2/10], Step [700/782], Loss: 0.9117
Epoch [3/10], Step [100/782], Loss: 1.1361
Epoch [3/10], Step [200/782], Loss: 0.9273
Epoch [3/10], Step [300/782], Loss: 0.8810
Epoch [3/10], Step [400/782], Loss: 0.9124
Epoch [3/10], Step [500/782], Loss: 0.7568
Epoch [3/10], Step [600/782], Loss: 0.9125
Epoch [3/10], Step [700/782], Loss: 0.8653
Epoch [4

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001
momentum = 0.9
weight_decay = 1e-4
dropout_rate = 0.3

# CIFAR-10 dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Learning rate scheduler
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 15], gamma=0.1)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    scheduler.step()

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Step [100/782], Loss: 1.9820
Epoch [1/100], Step [200/782], Loss: 1.7135
Epoch [1/100], Step [300/782], Loss: 1.6176
Epoch [1/100], Step [400/782], Loss: 1.4844
Epoch [1/100], Step [500/782], Loss: 1.6111
Epoch [1/100], Step [600/782], Loss: 1.6146
Epoch [1/100], Step [700/782], Loss: 1.3368
Epoch [2/100], Step [100/782], Loss: 1.5575
Epoch [2/100], Step [200/782], Loss: 1.3339
Epoch [2/100], Step [300/782], Loss: 1.4902
Epoch [2/100], Step [400/782], Loss: 1.2101
Epoch [2/100], Step [500/782], Loss: 1.1109
Epoch [2/100], Step [600/782], Loss: 0.9728
Epoch [2/100], Step [700/782], Loss: 1.0572
Epoch [3/100], Step [100/782], Loss: 1.3173
Epoch [3/100], Step [200/782], Loss: 1.2741
Epoch [3/100], Step [300/782], Loss: 1.4302
Epoch [3/100], Step [400/782], Loss: 1.2798
Epoch [3/100], Step [500/782], Loss: 1.4131
Epoch [3/100], Step [600/782], Loss: 1.2670
Epoch [3/100], Step [700/782], Loss: 1.1009
